# NLP - Natural Language Processing <h6>

## 1. spaCy
![](architecture-bcdfffe5c0b9f221a2f6607f96ca0e4a.svg)

As estruturas centrais dos dados no _spaCy_ são **Doc** e **Vocab**.

> **Doc** contém a sequencia de **tokens** e todas suas anotações

> **Vocab** contém _look-up_ _tables_ que faz uma informação comum entre todo documento

Centralizando _strings_, _word_ _vectors_ e atributos _lexical_, avita-se guardar multiplas cópias dos dados.

> O Objeto **Doc** é construido pelo **Tokenizer**, e então modificado por componentes do pipeline.


# 1.1. Container objects

> **Doc**  - Um container para acessar anotaçõess linguisticas

> **Span** - Um pedaço do objeto **Doc**

> **Token** - Um _token_ individual (palavra, pontuação, espaço, etc.)

> **Lexeme** - Uma entrada no vocabulario. Uma palavra sem contexto

# 1.2. Processing Pipeline

> **Language** - Pipeline de processamento textual. Carrega uma vez por processo e passa a instancia sobre a aplicação

> **Tokenizer** - Segmenta o text e cria objetos **Doc**

> **Lemmatizer** - Determina a forma base das palavras

> **Tagger** - Cria Tags de **part-of-speech** em **Doc**

> **DependencyParser** - Anota dependencias sintaticas em **Doc**

> **EntityRecognizer** - Anota Nomes de entidades em **Doc**

> **TextCategorizer** - Carrega categorias ou labels em **Doc**

> **Matcher** - Combina uma sequência de tokens, baseado em padrões

> **PhraseMatcher** - Combina sequência de tokens baseado em frases

> **EntityRuler** - Adiciona _spans_ de endidades ao **Doc** usando regras _token_ ou uma frase

> **Sentencizer** - Implementa uma logica de detecção de sentenças que não requer _dependency_ _parse_


## 1.3. Outras Classes

> **Vocab** - Uma tabela para acessar objetos **Lexeme**

> **StringStore** - Mapeia strings para _hash_ _values_

> **Vectors** Conteiner class for vector data keyed by string

> **GoldParse** - Coleção para _training_ _annotation__

> **GoldCorpus** - An annotated corpus, using the JSON file format. Manages annotations for tagging

# 2. Linguistic Features

## 2.1 Tokenization

Tokenizar significa fazer o splitting de um texto em segmentos com significados (**Tokens**)

O input do __Tokenizer__ é um texto unicode, a saída é um objeto __Doc__. Para construir um __Doc__, é necessário uma instância __Vocab__, uma sequencia de strings e, opcionalmente, uma sequencia de __spaces__ booleans, que permitem a manutenção entre __tokens__ na string original.

> A _tokenização_ realizada pelo spaCy é não-destrutiva, ou seja, é possivel sempre reconstruir o input original do output tokenizado

In [2]:
import spacy

nlp = spacy.load('pt_core_news_sm')

doc = nlp('Eu duvido que você tokeniza essa frase')

for token in doc:
    print(token.text)

Eu
duvido
que
você
tokeniza
essa
frase


![Processo de tokenizacao](tokenization-57e618bd79d933c4ccd308b5739062d6.svg)

### 2.1.1. Adicionando Regras de Tokenizacao



In [9]:
import spacy
from spacy.symbols import ORTH

nlp = spacy.load('pt_core_news_sm')

doc = nlp('Eu duvido que você tokeniza essa frase')

print([w.text for w in doc])

caso_especial = [{ORTH: 'toke'}, {ORTH: 'niza'}]
nlp.tokenizer.add_special_case('tokeniza', caso_especial)

print([w.text for w in nlp('tokeniza essa frase')])

['Eu', 'duvido', 'que', 'você', 'tokeniza', 'essa', 'frase']
['toke', 'niza', 'essa', 'frase']


### 2.1.2. Customizando a classe Tokenizer

Para criar um Tokenizer é necessário:
1.  Um dicionario de Casos Especiais
2. Uma função de __prefix_search__, para lidar com pontuações
3. Uma função de __suffix_search__, para lidar com pontuações sucessivas
4. Uma função de __infixes_finditer__, para lidar com separações que não são por espaço
5. Uma função __token_match__, para combinar strings que nunca farão split

### 2.1.3. Modificando regras existentes


As regras de __prefix__, __suffix__ e __infix__ podem ser modificadas.

> Tokenizer.suffix_search - Pode-se adicionar/remover regras

In [17]:
suffixes = nlp.Defaults.suffixes + (r'''-+$''',)
#print(suffixes)
suffix_regex = spacy.util.compile_suffix_regex(suffixes)
nlp.tokenizer.suffix_search = suffix_regex.search

In [19]:
suffixes = list(nlp.Defaults.suffixes)
suffixes.remove('\\[')
suffix_regex = spacy.util.compile_suffix_regex(suffixes)
nlp.tokenizer.suffix_search = suffix_regex.search

## 2.2 Sentece Segmentation

__spaCy__ utiliza análise de dependência para determinar os limites das sentenças. Isso significa que é necessário uma Modelagem Estatística.

### 2.2.1. Padrão: dependency parse

Para ver as sentenças de um __Doc__ usa-se __Doc.sents__

In [20]:
import spacy

nlp = spacy.load('pt_core_news_sm')

doc = nlp('Eu duvido que você tokeniza essa frase. Essa daqui nem se fala')

for sent in doc.sents:
    print(sent.text)


Eu duvido que você tokeniza essa frase.
Essa daqui nem se fala


### 2.2.2. Rule-based pipeline component

O componente __Sentencizer__ é um pipeline que splita as pontuações das sentenças.

Pode-se conectar no pipeline se precisar apenas da sentença entre os limites, sem realizar dependency parse

In [21]:
import spacy
from spacy.lang.en import English

nlp = English()
sentencizer = nlp.create_pipe('sentencizer')
nlp.add_pipe(sentencizer)
doc = nlp('This is a sentence. This is another sentence.')
for sent in doc.sents:
    print(sent.text)

This is a sentence.
This is another sentence.


### 2.2.3. Custom rule-based strategy

Pode-se criar um pipeline customizado que pega o __Doc__ e configura o atributo __Token.is_sent_start__ em cada token individual.

> Pode-se apenas configurar as limites antes de fazer o parse

In [33]:
import spacy

texto = 'Isso daqui é uma frase~~ eita outra frase ~~ isso daqui é outra frase'

nlp = spacy.load('pt_core_news_sm')
doc = nlp(texto)
print('Antes: ', [sent.text for sent in doc.sents])

def customiza_bondaries(doc):
    for token in doc[:-1]:
        if token.text == '~~~':
            doc[token.i+1].is_sent_start = True
    return doc

nlp.add_pipe(customiza_bondaries, before='parser')
doc = nlp(texto)
print('Depois: ', [sent.text for sent in doc.sents])

# naofunciona em portugues e nao sei pq

Antes:  ['Isso daqui é uma frase~~ eita outra frase ~~ isso daqui', 'é outra frase']
Depois:  ['Isso daqui é uma frase~~ eita outra frase ~~ isso daqui', 'é outra frase']


## 2.3. Combinação por regras (Token)

* Encontrar Frases, tokens e entidades

Rule_based matching possibilita encontrar palavras e frases dando acesso aos tokens e suas relações

### 2.3.1 Regras ou treinar um modelo

1. Para tarefas complexas, usualmente é melhor treinar um modelo estatístico de reconhecimento. Porém, modelos estatísticos requerem dados de treinamento.
2. Pode-se começar um projeto utilizando rule-based para ajudar a coletar os dados
3. Treinar modelo é util se há exemplos que você quer que seu sistema seja apto a generalizar.

## 2.3.2 Matching baseado em Token

o __Matcher__, que opera em __tokens__, serve como uma operação de rule-matching. As regras podem ser referidas por anotações em tokens. 

Para adicionar padrões:

1. Inicializar o __Matcher__ com um __vocab__.
2. Add __matcher.add()__ com um ID

In [37]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('pt_core_news_sm')
matcher = Matcher(nlp.vocab)
pattern = [{'LOWER': 'oi'}, {'IS_PUNCT': True}, {'LOWER': 'mundo'}]
matcher.add('OiMundo', None, pattern)

doc = nlp('Oi, mundo! Oi mundo')
matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(match_id, string_id, start, end, span.text)

951075153562793782 OiMundo 0 3 Oi, mundo


O Matcher retorna uma lista que mapeia o span em __doc[0:3]__

![](Captura.png)

## 2.4. Combinação por regra (Frases)

Usa-se __PhraseMatcher__ que cria um __Doc__ contendo multiplos tokens

In [40]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load('pt_core_news_sm')
matcher = PhraseMatcher(nlp.vocab)
terms = ["Barack Obama", "Angela Merkel", "Washington, D.C."]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", None, *patterns)

doc = nlp("German Chancellor Angela Merkel and US President Barack Obama "
          "converse in the Oval Office inside the White House in Washington, D.C.")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

Angela Merkel
Barack Obama
Washington, D.C.


Para criar patterns, cada frase deve ser processada com nlp. Se o modelo carregado faz um loop ou um list comprehension, pode ser ineficiente e lento.

Caso precisa-se apenas da tokenização e dos atributos Lexical, utiliza-se __nlp.make_doc__


### Combinação por formato

In [42]:
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

nlp = English()
matcher = PhraseMatcher(nlp.vocab, attr='Shape')
matcher.add('IP', None, nlp('127.0.0.1'), nlp('127.127.0.0'))

doc = nlp("Often the router will have an IP address such as 192.168.1.1 or 192.168.2.1.")
for match_id, start, end in matcher(doc):
    print("Matched based on token shape:", doc[start:end])

Matched based on token shape: 192.168.1.1
Matched based on token shape: 192.168.2.1


## 2.5. Combinação por reconhecimento de Entidade

O __EntityRuler__ é capaz de adicionar nome de entidades baseados em padrões de um dicionario.

> **Phrase patterns** para matches exatos: {'label': 'ORG', 'pattern': 'SENAI}

> **Token patterns** com um dicionario que descreve o token: {'label' : 'CIDADE', 'pattern': [{'LOWER':' 'Londrina'}, {'LOWER':'Ibipora'}]

In [56]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

nlp = English()
ruler = EntityRuler(nlp)
patterns = [{'label': 'ORG', 'pattern': 'SENAI'}, {'label' : 'CIDADE', 'pattern': 'Londrina'}, {'label' : 'CIDADE','pattern':'Ibipora'}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

doc = nlp('Couto gonna open a SENAI in Londrina and Ibipora')
print([(ent.text, ent.label_) for ent in doc.ents])

[('SENAI', 'ORG'), ('Londrina', 'CIDADE'), ('Ibipora', 'CIDADE')]
